In [1]:
import torch
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer, pipeline, CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
import requests

In [2]:
r = requests.get('https://sigongan-3f44b-default-rtdb.firebaseio.com/archive.json').json()
list = [value for key, value in r.items()]
df = pd.DataFrame(list)

In [3]:
from g_spread import *
gsp = GSpread()

In [4]:
gsp.updateByDf(df[['imageUrl', 'description']], [4,2])

In [5]:
image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
df['captioning'] = df.apply(lambda x: image_to_text(x.imageUrl)[0]['generated_text'], axis = 1)

/Users/jusang/Library/Python/3.9/lib/python/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
/Users/jusang/Library/Python/3.9/lib/python/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [11]:
gsp.updateByDf(df[['captioning']], [4, 5])

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [ ]:
def classifier1(url, label=['real photo', 'document']):
    image = Image.open(requests.get(url, stream=True).raw)
    inputs = processor(text=label, images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1) #
    return torch.max(probs, dim=1).indices.item()

In [ ]:
df['class1'] = df.imageUrl.apply(lambda x: classifier1(x, label=['picture', 'document']))

In [ ]:
gsp.updateByDf(df[['class1']], [4, 4])

## QA Chatbot

In [7]:
import os
import openai
from dotenv import load_dotenv
import re
import time

In [8]:
load_dotenv()
openai.api_key = os.getenv('API_KEY')

params = {
    'engine': 'text-davinci-003',
    'max_tokens': 1000,
    'temperature': 0.1
}

In [1]:
context = ''
imageUrl = "https://firebasestorage.googleapis.com/v0/b/sigongan-3f44b.appspot.com/o/images%2Fscaled_1678243980217.jpg?alt=media&token=76d08213-9929-4f73-bc77-c48ffd639079"

print("Welcome to sigongan-ai v0.0.1.\n")

while True:
    # Select mode
    print("Please select the mode\n1: Brief description 2: QA mode 3: Quit")
    mode = input()
    if ('1' not in mode or '2' not in mode or '3' not in mode):
        print(f"{mode} isn't a selectable mode. Try again")
        continue
    
    if('3' in mode): break
    
    # mode 1
    if('1' in mode):
        while True:
            print('You selected mode 1. Preparing for description...')
            print('"Description"')
            continue
        
        # mode 2
    elif(mode in '2'):
        while True:
            print(f'Question: \n')
            question = input()
            prompt = f"Question: {question}. Plase give me the most detailed answer for this question. Answer:"
            response = openai.Completion.create(engine=params['engine'], prompt=prompt,temperature=params['temperature'], max_tokens=params['max_tokens'])
            answer = response.choices[0].text.strip()
            print(f"Answer: {answer}\n Ask again: 1\nQuit: else")
            mode = input()
            if ('1' in mode): continue
            else: break            
    
    
    
    
    

Welcome to sigongan-ai v0.0.1.

Please select the mode
1: Brief description 2: QA mode 3: Quit
 isn't a selectable mode. Try again
Please select the mode
1: Brief description 2: QA mode 3: Quit
 isn't a selectable mode. Try again
Please select the mode
1: Brief description 2: QA mode 3: Quit
 isn't a selectable mode. Try again
Please select the mode
1: Brief description 2: QA mode 3: Quit
3 isn't a selectable mode. Try again
Please select the mode
1: Brief description 2: QA mode 3: Quit
3 isn't a selectable mode. Try again
Please select the mode
1: Brief description 2: QA mode 3: Quit
3 isn't a selectable mode. Try again
Please select the mode
1: Brief description 2: QA mode 3: Quit
1 isn't a selectable mode. Try again
Please select the mode
1: Brief description 2: QA mode 3: Quit
 isn't a selectable mode. Try again
Please select the mode
1: Brief description 2: QA mode 3: Quit
 isn't a selectable mode. Try again
Please select the mode
1: Brief description 2: QA mode 3: Quit
e isn't a 

KeyboardInterrupt: Interrupted by user